In [4]:
import spacy
from spacy.training.example import Example

In [5]:
nlp = spacy.load('en_core_web_sm')

In [8]:
to_analyze = ('Hello code & Supply,'
       'my name ins Josh and tonight'
       'we\' re in Pittsburgh')

In [9]:
doc=nlp(to_analyze)

In [10]:
ents = [(x.text,x.label_) for x in doc.ents]
print(ents)
ents

[('Josh', 'PERSON'), ('Pittsburgh', 'GPE')]


[('Josh', 'PERSON'), ('Pittsburgh', 'GPE')]

In [11]:
# Setup the phrase matches

In [14]:
from spacy.matcher import PhraseMatcher 
import random 

In [39]:
label = "CIADIR"
matcher = PhraseMatcher(nlp.vocab)
for i in ['Gina Haspel', 'Gina', 'Haspel']:
    matcher.add(label,None,nlp(i))


In [40]:
one = nlp('Gina Haspel was nominated in 2018')
matches =  matcher(one)
[match for match in matches] # reprentation of label, to extract information from 
# this, we will we the utility function

[(17539557946531887154, 0, 1),
 (17539557946531887154, 0, 2),
 (17539557946531887154, 1, 2)]

This function convert the output of the PhraseMacher to something usable in training. The training data needs a string index of characters(start, end, label) while the matched output uses index of words from nlp document. 

In [41]:
def offseter(lbl, doc, matchitem):
    o_one = len(str(doc[0:matchitem[1]]))
    subdoc = doc[matchitem[1]:matchitem[2]]
    o_two = o_one + len(str(subdoc))
    return (o_one, o_two,lbl)

## Setup the phrase matches

In [167]:
label = "Cov"
matcher = PhraseMatcher(nlp.vocab)
for i in ['test','Covid-19', 'Laboratories']:
    matcher.add(label,None,nlp(i))


## Gather training data 

In [168]:
res = []
to_train_ents = []

with open('text.txt') as gh:
    line =  True
    compt = 0
    while line: 
        line =  gh.readline()
        mnlp_line = nlp(line)
        matches = matcher(mnlp_line)
        res = [offseter(label, mnlp_line,x)
              for x in matches]
        to_train_ents.append((line, dict(entities = res)))
        
        

## Train the Recognizer

In [169]:
optimizer = nlp.begin_training()

other_pipes = [pipe for pipe in nlp.pipe_names if pipe !='ner']


In [170]:
other_pipes

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer']

In [174]:
with nlp.disable_pipes(*other_pipes): # only train NER
    for itn in range(20): # we are going to go throught the training data 20 times
        losses = {}
        random.shuffle(to_train_ents) # we shuffle the data 
        for item in to_train_ents:
            example = Example.from_dict(nlp.make_doc(item[0]), item[1])
            nlp.update([example],
            sgd=optimizer,
            drop = 0.35,
            losses =  losses)
            #print("Ner",losses["ner"])
            # this update function Takes 
                       # the model we updated at the begininig at update it with the new information
                       #about the tokens, the position of the tokens and the string.
            

ValueError: too many values to unpack (expected 3)

In [164]:
one = nlp(' data test-to-treat Laboratories program stymied by lack of services, tech problems')
matches =  matcher(one)
[match for match in matches] # reprentation of label, to extract information from 
# this, we will we the utility function

[(16114187870790477818, 2, 3), (16114187870790477818, 7, 8)]

In [165]:
for entity in one.ents:
    print(entity,entity.label_)

In [166]:
for text,_ in to_train_ents:
    doc = nlp(text)
    result = [(ent,ent.label_) for ent in doc.ents]
    print(result)

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [97]:
to_train_ents[1]

('Efforts to streamline delivery of at-home test results\n',
 {'entities': [(33, 40, 'Covid')]})

In [120]:
to_train_ents

[("States like Tennessee encourage people who use at-home tests to submit their results to the manufacturers so they can be tracked. It's unclear how many people actually do that, but there are several efforts underway to track home tests better.\n",
  {'entities': [(46, 53, 'Covid')]}),
 ('Efforts to streamline delivery of at-home test results\n',
  {'entities': [(33, 40, 'Covid')]}),
 ('The Association of Public Health Laboratories announced in March that it was awarded the NIH contract, limited to $8.8 million over three years, to work on improving the reporting of over-the-counter at-home Covid-19 test results to public health officials. The contract employs AIMS to collect data from at-home test manufacturers and then distribute the data to states and jurisdictions.\n',
  {'entities': [(32, 44, 'Covid'),
    (199, 206, 'Covid'),
    (207, 215, 'Covid'),
    (304, 311, 'Covid')]}),
 ('Another platform pushing for more efficiency is ReportStream, a cloud-based data routing system se

In [172]:
TRAIN_DATA[1:3]

NameError: name 'TRAIN_DATA' is not defined